In [ ]:
%pip install -Uqq langchain-weaviate

In [ ]:
%pip install weaviate-client langchain langchain-huggingface

In [ ]:
import weaviate
from langchain_weaviate.vectorstores import WeaviateVectorStore

# docker run -p 8080:8080 -p 50051:50051 cr.weaviate.io/semitechnologies/weaviate:1.28.3


In [ ]:
from langchain.vectorstores import Weaviate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import weaviate

In [ ]:
# Step 1: Initialize Hugging Face Embedding Model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")  # Replace with your desired model

# Step 2: Load PDF and Split into Chunks
loader = PyPDFLoader("../../00-example_data/layout-parser-paper.pdf")
pdf_docs = loader.load()

print(f"Loaded {len(pdf_docs)} documents from the file.")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
documents = text_splitter.split_documents(pdf_docs)



In [7]:
weaviate_client = weaviate.connect_to_local()

weaviate_store = WeaviateVectorStore.from_documents(documents, embedding_model, client=weaviate_client)



In [ ]:

# Step 5: Perform a Query
query = "What algorithm is discussed in the document?"
results = weaviate_store.similarity_search(query, k=3)

print("\nMost Similar Documents:")
for idx, result in enumerate(results, start=1):
    print(f"{idx}. {result.page_content}")
